# Crawler Noticias



## Introdução

Estudo projeto de construção de um bot crawler para extrair e indexar noticias de sites.

É recomendavel baixar uma única vez o html do site para reduzir a necessidade de acessar o site várias vezes para cada processamento reduzindo tempo e custo computacionais para máquina local e servidor.

Usamos módulo _Requests_ para baixar o html da página a ser processada e com módulo _BeautifulSoup_ processamos o conteúdo html buscando por conteúdos especificos. Ao inspecionar o código HTML verificamos em quais blocos estão as principais noticias - em geral em tags como "section", "h1"/"h2"/"h3" e "a" - considerando o seletor css que especifica cada bloco de interesse afim de refinar a busca e garantir que serão extraindos os trechos relevantes.

Sites com boas práticas de HTML/CSS tendem a serem mais faceis de serem processados e indexados por mecanismos de buscas facilitando os usuários em encontrar seus conteúdos em pesquisas aumentando assim o engajamento em suas plataformas contribuindo para seu melhor desenvolvimento.

Neste presente momento o projeto está considerando três portais de notícias que são _BBC Brasil_, _CNN Brasil_ e _G1/Globo_. 

Créditos e direitos reservados às referentes plataformas mencionadas.

(Obs.: Conteúdo livre com fins informativos e de divulgação. )


## Módulos

Abaixo _import_ dos módulos que contém os recursos usados.

* BeautifulSoup
* Requests
* re (regular expression)
* crawlernewsg1 (acessa site e extrai lista de principais noticias na página principal do site)
* IPython.core.display -> display, HTML (renderiza conteúdos html )

In [8]:
from bs4 import BeautifulSoup
import requests
import re
from crawlernewsg1 import *
import random

# Render html content
from IPython.core.display import display, HTML

/tmp/ipykernel_33177/1468998341.py:8: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Modelagem Portal G1



Capturando noticias do portal G1 da Globo
Funções básicas. Especifincando "selector css" classe 'bastian-page'  para tags div.

In [9]:
# Baixando html do portal de noticia para processamento e salvando em disco.
url = 'https://g1.globo.com'
attr = {'class': 'bastian-page'}
data = g1_(requests.get( url ).content, 'div', attr, debug=False)
dw = requests.get('https://g1.globo.com/').content
with open('/tmp/data', 'w') as fl:
    fl.write( dw.decode() )
    

In [10]:
type(data)

tuple

In [11]:
# Replace field name 'titulo' for 'title' name.
#dw = [ { key.replace('titulo', 'title'):value for key, value in i.items() } for i in dw ]
#dw = [ { key.replace('url', 'href'):value for key, value in i.items() } for i in dw ]
#replKey = lambda listdc, old_key, new_key: [ ]
def replKey(listdc, old_key, new_key  ):
    dt = []
    
    
    for i in listdc:
        for keys, value in i.items():
            dt.append( { keys.replace('url', 'href'): value } ) 
            
    return dt

data = replKey(data[0], 'url', 'href' )

print(data)

[{'title': 'Mensagens e prints ligam jogadores a esquema de manipulação'}, {'href': 'https://g1.globo.com/go/goias/noticia/2023/05/13/chamadas-de-video-com-apostador-comprovantes-de-pagamento-e-mensagens-veja-provas-apontadas-pelo-mp-que-ligam-jogadores-a-esquema-de-manipulacao-de-jogos.ghtml'}, {'title': 'Preso acusado manipular jogos ligou da cadeia para esposa; VEJA'}, {'href': 'https://g1.globo.com/go/goias/noticia/2023/05/13/preso-acusado-de-integrar-grupo-que-manipulava-resultados-de-jogos-ligou-para-a-esposa-de-dentro-da-cadeia-e-pediu-dinheiro-para-comprar-celular-veja-conversas.ghtml'}, {'title': 'Treinador de futsal denunciado por assediar jogadoras morre na cadeia'}, {'href': 'https://g1.globo.com/ce/ceara/noticia/2023/05/13/treinador-de-futsal-denunciado-por-assediar-ao-menos-12-jogadoras-morre-dentro-de-presidio-apos-passar-mal-no-ceara.ghtml'}, {'title': 'VÍDEO mostra momento em que meteoro é engolido por nuvem em SC'}, {'href': 'https://g1.globo.com/sc/santa-catarina/not

In [12]:
dw, cwn = g1_( dw, 'div', attr)

Ao abrir link da noticia pesquisar pela tag *'p'* com atributos **class** com valor **"content-text__container** definir o tamanho para caso extrair só parte do corpo do texto.


Retornando uma lista dos itens encontrados (como visto acima no código) pegamos estes itens e os concatenamos exibintido o texto no corpo da noticia. (_Como visto abaixo_)

Cada "_evt" (_css selector_ class) class css em "bastian-page" refere-se a uma noticia na lista central de noticias.
Dentro de cada "_evt" haverá "bastian-feed-item" e neste o feed-post. 

**feed-post-body** _contêm_  ( 'feed-post-link', 'feed-post-body-title', 'feed-post-body-resumo')

**bastian-feed-item** _contem_ um feed-post-body referindo-se a cada item (noticia)


Para link da noticia (quando acessando a noticia)

**content-head__title** em tag 'h1' (Título da noticia)

**content-head__subtitle** em tag 'h2' (subtitulo/resumo da noticia)

**content-text__container** corpo do texto da noticia css-selector, tag 'p' (pegar só a primeira referente ao primeiro paragrafo da noticia)

In [13]:
news2 = '<h1 style="padding: 12px;">Notícias</h1>'
news2 += '<br><br>'.join( [ '<br>'.join( [ str( i['title'] ) , str( i[ 'url' ]  ) ] ) for i in dw ] )
display( HTML( news2 ))

# CNN Crawler de Noticias do Portal

In [14]:
cnn_data = requests.get('https://www.cnnbrasil.com.br/')


In [15]:
cnn_soup = BeautifulSoup(cnn_data.content, 'html.parser')
cnn_nw_data = cnn_soup.find_all('section')
cnn_nw_data[0]

<section class="hot__content"> <div class="carousel"> <div class="carousel__screen infinite"> <ul class="carousel__track"> <li class="carousel__item"><div class="hot__list__itens"> <a href="https://www.cnnbrasil.com.br/politica/a-cnn-bolsonaro-diz-que-vai-processar-lula-por-falas-sobre-mansao-e-mortes-na-pandemia/" target="_self" title="LULA X BOLSONARO À CNN, Bolsonaro diz que vai processar Lula por falas sobre mansão e mortes na pandemia"> <img class="tb" src="https://www.cnnbrasil.com.br/wp-content/uploads/sites/12/Reuters_Direct_Media/BrazilOnlineReportTopNews/tagreuters.com2022binary_LYNXMPEI9T0AZ-FILEDIMAGE.jpg?w=65&amp;h=37&amp;crop=1" title="LULA X BOLSONARO À CNN, Bolsonaro diz que vai processar Lula por falas sobre mansão e mortes na pandemia"/> <div class="i_ch"> <i></i> <span class="ch"> LULA X BOLSONARO </span> </div> <span class="tp"> À CNN, Bolsonaro diz que vai processar Lula por falas sobre mansão e mortes na pandemia </span> </a></div></li><li class="carousel__item"><

In [16]:
c = cnn_nw_data[0].find('a')
#dir(c)


In [17]:
c.attrs

{'href': 'https://www.cnnbrasil.com.br/politica/a-cnn-bolsonaro-diz-que-vai-processar-lula-por-falas-sobre-mansao-e-mortes-na-pandemia/',
 'title': 'LULA X BOLSONARO À CNN, Bolsonaro diz que vai processar Lula por falas sobre mansão e mortes na pandemia',
 'target': '_self'}

In [18]:
cnn_list_news = []
aux = None 
for news in cnn_nw_data:
    aux = news.find('a')
    
    
    try :
        
        aux = aux.attrs
        cnn_list_news.extend( [ { 'title': aux['title'], 'href': aux['href'] } ] ) 

    except:
        pass

print(cnn_list_news)

[{'title': 'LULA X BOLSONARO À CNN, Bolsonaro diz que vai processar Lula por falas sobre mansão e mortes na pandemia', 'href': 'https://www.cnnbrasil.com.br/politica/a-cnn-bolsonaro-diz-que-vai-processar-lula-por-falas-sobre-mansao-e-mortes-na-pandemia/'}, {'title': 'Governo espera ter regulamentação das apostas online até o fim do ano, diz Fazenda', 'href': 'https://www.cnnbrasil.com.br/nacional/governo-espera-ter-regulamentacao-das-apostas-online-ate-o-fim-do-ano-diz-fazenda/'}, {'title': 'Se o dinheiro está caro, a culpa não é do BC, porque é malvado, mas do governo, que deve muito, diz Campos Neto à CNN', 'href': 'https://www.cnnbrasil.com.br/economia/se-o-dinheiro-esta-caro-a-culpa-nao-e-do-bc-porque-e-malvado-mas-do-governo-que-deve-muito-diz-campos-neto-a-cnn/'}, {'title': 'Política', 'href': 'https://www.cnnbrasil.com.br/politica/'}, {'title': 'Pop', 'href': 'https://www.cnnbrasil.com.br/pop/'}, {'title': 'Mercado', 'href': '/cotacoes/bolsa'}, {'title': 'Economia', 'href': 'htt

In [19]:
cnn_list_news_ma = None
for i in cnn_list_news:
    
    try:
        print(f"\n{i['title']} \n{i['href']}\n\n" ) 
    except:
        pass 
    #keys = i.keys()
    #print(keys)   
    

    #print( f'\t\n\n{out}') 
    #out = '' 



LULA X BOLSONARO À CNN, Bolsonaro diz que vai processar Lula por falas sobre mansão e mortes na pandemia 
https://www.cnnbrasil.com.br/politica/a-cnn-bolsonaro-diz-que-vai-processar-lula-por-falas-sobre-mansao-e-mortes-na-pandemia/



Governo espera ter regulamentação das apostas online até o fim do ano, diz Fazenda 
https://www.cnnbrasil.com.br/nacional/governo-espera-ter-regulamentacao-das-apostas-online-ate-o-fim-do-ano-diz-fazenda/



Se o dinheiro está caro, a culpa não é do BC, porque é malvado, mas do governo, que deve muito, diz Campos Neto à CNN 
https://www.cnnbrasil.com.br/economia/se-o-dinheiro-esta-caro-a-culpa-nao-e-do-bc-porque-e-malvado-mas-do-governo-que-deve-muito-diz-campos-neto-a-cnn/



Política 
https://www.cnnbrasil.com.br/politica/



Pop 
https://www.cnnbrasil.com.br/pop/



Mercado 
/cotacoes/bolsa



Economia 
https://www.cnnbrasil.com.br/economia/



Internacional 
https://www.cnnbrasil.com.br/internacional/



Viagem & Gastronomia 
https://www.cnnbrasil.co

In [20]:
len(dw)

7

In [21]:
len(cnn_list_news)

15

# BBC Brasil 

Crawler das notícias do portal do site BBC Brasil.

In [22]:
url_bbc = 'https://www.bbc.com/portuguese'
url_bbc_base = 'https://www.bbc.com'

In [23]:
bbc_data = requests.get(url_bbc)
bbc_soup = BeautifulSoup(bbc_data.content, 'html.parser')
bbc_sections =  bbc_soup.find_all('section' , class_= 'bbc-iinl4t')

In [24]:
bbc_news_lists = []

``` python
url_bbc_base+bbc_sections[0].select('a')[0]['href']
```

Saída 'https://www.bbc.com/portuguese/brasil-63507138'

In [25]:
bbc_sections[0].select('a')[0].text

'As sacerdotisas africanas perseguidas pela Inquisição no Brasil'

In [26]:
for news in bbc_sections:
    bbc_news_lists.extend( [ { 'title': i.text, 'url': i['href'] } for i in news.select('a') ] )

In [27]:
bbc_news_lists = []
aux = None 

for section_news in bbc_sections:
    for news in section_news.select( 'a' ):
        if news['href'].find('topic') != -1:
            pass
        else :
            if news['href'].find('https') != -1:
                bbc_news_lists.extend( [ { 'title': news.text, 'href': news['href'] } ] )
            else:
                bbc_news_lists.extend( [ { 'title': news.text, 'href': url_bbc_base+news['href'] } ] )

In [28]:
print(f'Número de notícias: {len(bbc_news_lists)}')

Número de notícias: 63


In [29]:
random.choices(bbc_news_lists, k = 3)

[{'title': 'O que o SUS está ensinando ao serviço de saúde britânico',
  'href': 'https://www.bbc.com/portuguese/articles/cq5ww8jryk0o'},
 {'title': 'Vídeo, O desastre que ameaça líder turco, no poder há duas décadas, Duration 4,54',
  'href': 'https://www.bbc.com/portuguese/internacional-65573941'},
 {'title': 'A família boêmia que chocou a Grã-Bretanha vitoriana',
  'href': 'https://www.bbc.com/portuguese/articles/cjqk08z5ew4o'}]

In [30]:
print(url_bbc_base+'/portuguese/topics/cz74k71p8ynt')

https://www.bbc.com/portuguese/topics/cz74k71p8ynt


In [31]:
with open('bbc_news.txt', 'w') as fl:
    for i in bbc_news_lists:
        fl.write(i['title'] +'\n' + i['href'] +'\n\n')
    fl.close()

# Portal Band

Busca de lista de notícias do portal Band.

In [32]:
url_band = 'https://www.band.uol.com.br/'

In [33]:
band_data = requests.get(url_band).content
soup_band = BeautifulSoup(band_data, 'html.parser')


In [34]:
news_band_section = soup_band.find_all('section')
print(f'Número: {len(news_band_section)}')

Número: 17


In [35]:
aux = []
list_ = None
for secs in news_band_section:
    list_ = secs.find_all('a', class_='link')
    for i in list_:
        
        try:
            aux.extend( [ { 'title': i.h3.text+'. '+ i.h2.text, 'href': i['href']} ] )
        except:
            aux.extend( [ { 'title': i.text, 'href': i['href'] } ] )
    #aux.extend([ { 'title': i}  for i in list_ ]  )
news_list_band = [] 
news_list_band.extend(random.choices(aux, k=4 ) )

In [36]:
n = soup_band.find_all('a')

In [37]:
import random

random.choices(news_list_band, k = 4)[0:]
for i in news_list_band:
    print(f"\n{i['title']}\n{i['href']}")


Antonio Fagundes precisou tirar carteira de motorista especial para Carga Pesada
/entretenimento/melhor-da-tarde/noticias/antonio-fagundes-precisou-tirar-carteira-de-motorista-especial-para-carga-pesada-16601873

Luiza Martins relembra episódio engraçado com Marília Mendonça: “Não enxergava” 
/entretenimento/faustao-na-band/noticias/luiza-martins-relembra-episodio-engracado-com-marilia-mendonca-nao-enxergava-16601910

TRAGÉDIA
/noticias/cenipa-conclui-relatorio-sobre-acidente-aereo-que-matou-marilia-mendonca-16601833

Show do Esporte
/esportes/show-do-esporte


In [38]:
n[0]

<a _ngcontent-sc85="" aria-label="Band" class="logo logo--portal logo-name--Band logo--image center" href="/"><img _ngcontent-sc85="" alt="Band" class="logo-image" height="35" src="https://pubimg.band.uol.com.br/Files/logo-band-2022-v2.png?v2" width="101"/><!-- --><!-- --><!-- --></a>

# Montando Lista de Notícias

In [39]:
number_news = 6


In [40]:
import random

In [41]:
news_list = []

news_list.extend( [ {'title': random.choices( [ (news['title'], news['url']) for news in dw ] , k = 2 ), 'source': 'G1/Globo' } ] )

news_list.extend( [ {'title': random.choices( [ (news['title'], news['href']) for news in cnn_list_news ] , k = 2 ), 'source': 'CNN Brasil' } ] )

news_list.extend( [ {'title': random.choices( [ (news['title'], news['href']) for news in bbc_news_lists ] , k = 2 ), 'source': 'BBC Brasil' } ] )


In [42]:
for i in news_list:
    for j in i['title']:
        print(f"{j[0]}. \n{j[1]}\n")
    
    print(f"Fonte: {i['source']}\n\n")

Quem é Danilo Tandera, o miliciano mais procurado do RJ . 
https://g1.globo.com/rj/rio-de-janeiro/noticia/2023/05/13/quem-e-danilo-tandera-miliciano-mais-procurado-do-rj-e-com-apenas-duas-passagens-rapidas-pela-cadeia.ghtml

Treinador de futsal denunciado por assediar jogadoras morre na cadeia. 
https://g1.globo.com/ce/ceara/noticia/2023/05/13/treinador-de-futsal-denunciado-por-assediar-ao-menos-12-jogadoras-morre-dentro-de-presidio-apos-passar-mal-no-ceara.ghtml

Fonte: G1/Globo


Se o dinheiro está caro, a culpa não é do BC, porque é malvado, mas do governo, que deve muito, diz Campos Neto à CNN. 
https://www.cnnbrasil.com.br/economia/se-o-dinheiro-esta-caro-a-culpa-nao-e-do-bc-porque-e-malvado-mas-do-governo-que-deve-muito-diz-campos-neto-a-cnn/

Ciência e Tecnologia. 
https://www.cnnbrasil.com.br/tecnologia/

Fonte: CNN Brasil


Podcast busca entender como brasileiros chegaram a atual grau de divisão; ouça aqui. 
https://www.bbc.com/portuguese/podcasts/p0cyhvny

Facebook. 
https://

## Salvando Todas Noticias (JSON)

### View news for test algorithm

In [43]:
# List news
news = None
portaisvar = [
    'dw', #Globo/G1
    'cnn_list_news', # CNN Brazil
    'bbc_news_list' # BBC Brazil
]

portais = { 'dw': 'Globo/G1', 'cnn_list_news': 'CNN Brasil', 'bbc_news_lists': 'BBC Brasil' }    

In [44]:
dw[0].keys(), cnn_list_news[0].keys(), bbc_news_lists[0].keys()

(dict_keys(['title', 'url']),
 dict_keys(['title', 'href']),
 dict_keys(['title', 'href']))

In [45]:
# Replace field name 'titulo' for 'title' name.
dw = [ { key.replace('titulo', 'title'):value for key, value in i.items() } for i in dw ]
dw = [ { key.replace('url', 'href'):value for key, value in i.items() } for i in dw ]
dw

[{'title': 'Mensagens e prints ligam jogadores a esquema de manipulação',
  'href': 'https://g1.globo.com/go/goias/noticia/2023/05/13/chamadas-de-video-com-apostador-comprovantes-de-pagamento-e-mensagens-veja-provas-apontadas-pelo-mp-que-ligam-jogadores-a-esquema-de-manipulacao-de-jogos.ghtml'},
 {'title': 'Preso acusado manipular jogos ligou da cadeia para esposa; VEJA',
  'href': 'https://g1.globo.com/go/goias/noticia/2023/05/13/preso-acusado-de-integrar-grupo-que-manipulava-resultados-de-jogos-ligou-para-a-esposa-de-dentro-da-cadeia-e-pediu-dinheiro-para-comprar-celular-veja-conversas.ghtml'},
 {'title': 'Treinador de futsal denunciado por assediar jogadoras morre na cadeia',
  'href': 'https://g1.globo.com/ce/ceara/noticia/2023/05/13/treinador-de-futsal-denunciado-por-assediar-ao-menos-12-jogadoras-morre-dentro-de-presidio-apos-passar-mal-no-ceara.ghtml'},
 {'title': 'VÍDEO mostra momento em que meteoro é engolido por nuvem em SC',
  'href': 'https://g1.globo.com/sc/santa-catarina/

In [46]:
#[ { key.replace('title', 'titulo' ) } ]
newslist = dw + cnn_list_news + bbc_news_lists
#newslist

In [47]:
import os
count = 0
try:
    os.mkdir( 'newsdata' )
except FileExistsError:
    print("File exist")
    
webdata = ''
# Save data of link news in files to folder.
for news in newslist:
    try:
        #news['data'] = requests.get(news['href']).text
        foldername = f'newsdata'
        #os.mkdir( foldername )
        news['datafile'] = f'{foldername}/'+str( count )
        with open( foldername+f'/{str(count)}', 'a' ) as fl:
            webdata = requests.get( news['href'] ).text 
            fl.write( webdata )
            fl.close()
            
        count += 1
    except :
        print('not get page')
        continue
    

File exist
not get page


In [48]:
newslist[0]['datafile']

'newsdata/0'

In [49]:
p = requests.get( newslist[0]['href'] )
soup = BeautifulSoup( p.text, 'html.parser')
texto = soup.get_text()
print( texto )

 Chamadas de vídeo com apostador, comprovantes de pagamento e mensagens: veja provas apontadas pelo MP que ligam jogadores a esquema de manipulação de jogos | Goiás | G1

                   Goiás                          








fique por dentro


          Imposto de Renda 
      

          Mega-Sena
      

          Quiz
      

          Frio
      

          Anderson Torres solto
      






Empresário Bruno Lopez de Moura que foi preso em São Paulo acusado de participar de esquema criminoso — Foto: Reprodução/Instagram     Empresário Bruno Lopez de Moura que foi preso em São Paulo acusado de participar de esquema criminoso — Foto: Reprodução/Instagram        Também havia o "Núcleo Financiadores". Eles eram os responsáveis por assegurar a existência de verbas para o pagamento dos jogadores aliciados e também nas apostas manipuladas.       Além disso, havia o "Núcleo Intermediadores". Estes eram responsáveis por indicar contatos e facilitar a aproximação entre apostadores e atl

In [50]:
d = ''
with open(foldername+'/0', 'r') as fl:
    d = fl.read()
    fl.close()


soup = BeautifulSoup(d, 'html.parser')
#souptext = soup.text
content = soup.find_all('div')
#dir(soup)

In [51]:
texto = ' '.join( [ i.get_text() for i in content ] )
texto = texto.replace('\n', ' ')
with open('texto', 'w' ) as fl:
    #fl.write( souptext )
    fl.write( texto )
    

In [52]:
print( texto[0:100] )

              Goiás                                   fique por dentro             Imposto de Renda 


In [53]:
import json

with open('noticias.json', 'w') as fl:
    fl.write(json.dumps(newslist, ensure_ascii=False, indent=4))
    fl.close()

### Salva noticias

In [54]:
def savenews(newslist: list, filename: str) -> None:
    import json
    jsondata = None
    with open(filename, 'w') as fl:
        fl.write( json.dumps( newslist, encode='utf-8', indent=4 ) )
        fl.close()
    print('Saved...')
    

In [55]:
i = j = None

for i in news_list:
    for j in i['title']:
        print(f'{j[0]}.')
    print(f'Fonte: {i["source"] }\n\n ' )

Quem é Danilo Tandera, o miliciano mais procurado do RJ .
Treinador de futsal denunciado por assediar jogadoras morre na cadeia.
Fonte: G1/Globo

 
Se o dinheiro está caro, a culpa não é do BC, porque é malvado, mas do governo, que deve muito, diz Campos Neto à CNN.
Ciência e Tecnologia.
Fonte: CNN Brasil

 
Podcast busca entender como brasileiros chegaram a atual grau de divisão; ouça aqui.
Facebook.
Fonte: BBC Brasil

 


In [56]:
i = j = None

print(f"Olá bem vindo ao Diário de Notícias Dimensão Alfa. Estas são as principais manchetes do dia.\n")
for i in news_list:
    print(f'Portal de Notícias {i["source"] }\n ' )
    for j in i['title']:
        print(f'{j[0]}.')
    print('\n\n')

Olá bem vindo ao Diário de Notícias Dimensão Alfa. Estas são as principais manchetes do dia.

Portal de Notícias G1/Globo
 
Quem é Danilo Tandera, o miliciano mais procurado do RJ .
Treinador de futsal denunciado por assediar jogadoras morre na cadeia.



Portal de Notícias CNN Brasil
 
Se o dinheiro está caro, a culpa não é do BC, porque é malvado, mas do governo, que deve muito, diz Campos Neto à CNN.
Ciência e Tecnologia.



Portal de Notícias BBC Brasil
 
Podcast busca entender como brasileiros chegaram a atual grau de divisão; ouça aqui.
Facebook.





# Sobre

## Dimensão Alfa

Dimensão Alfa projetos e conteúdos de tecnologia.

## Info

O presente projeto tem sido usado com fins de divulgação e facilitação de acesso a noticias e conhecimento em comunhão com objetivo da plataforma/página Dimensão Alfa. 
Conteúdos de terceiros são de responsabilidades dos mesmos bem como seus direitos autorais.

O projeto encontra-se em desenvolvimento, inicialmente fôra batizado de Ani Fátima Liu, e estará passando por alterações estando de inicio disponibilizado em formato "_jupyter notebook_" podendo servir como _case_ de estudo para os que se interessam por "web scrap" (raspagem de dados).

Tecnologias foram usadas para gerar vídeo de noticias diária para página [Dimensão Alfa no facebook](https://www.facebook.com) e [Youtube](https://www.youtube.com/@dimensaoalfa); foi usada as seguintes tecnologias:

* [Editor de códigos VSCode](https://code.visualstudio.com/)
* [Python (linguagem de programação)](https://www.python.org/)
* [Ambiente JupyterLab](https://jupyter.org/)
* [Biblioteca "Requests"](https://requests.readthedocs.io/en/latest/)
* [Biblioteca "BeautifulSoup"](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
* [Serviço de Sintese de Voz Microsoft/Azure](https://speech.microsoft.com)


Peço e agradeço a compreensão e apoio de todos. 

Para contribuições, dúvidas, sugestões visitem nossas páginas no [Facebook](https://www.facebook.com/).


## Sugestões de Conteúdo

Deixamos abaixo algumas sugestões de conteúdos e canais com recursos para estudos e pesquisa que podem ser uteis para quem se interessa por tecnologia, programação de computadores, matemática, ciências de dados e inteligência artificial.

* [Programação Dinâmica](https://www.youtube.com/c/Programa%C3%A7%C3%A3oDin%C3%A2mica)
* [Toda Matemática](https://www.youtube.com/c/GustavoViegascurso)
* [Matemática Universitária](https://www.youtube.com/c/Matem%C3%A1ticaUniversit%C3%A1riaProfRenan)
* [Reflexões Matemáticas](https://www.youtube.com/c/Reflex%C3%B5esMatem%C3%A1ticasDrDilbertoJ%C3%BAnior)
* [Programação Descomplicada](https://www.youtube.com/user/progdescomplicada)
* [Univesp](https://www.youtube.com/user/univesptv)
* [USP no Youtube](https://www.youtube.com/c/CanalUSP)
* [IME/USP](https://www.ime.usp.br/)
* [IMPA](https://www.youtube.com/c/impabr)



## Links

* [Dimensão Alfa](https://www.dimensaoalfa.com.br)
* [Facebook](https://www.facebook.com/dimensaoalfa)
* [Youtube](https://www.youtube.com/@dimensaoalfa)
* [WSRicardo](https://wsricardo.blogspot.com)